In [1]:
from ceruleanml import data
from ceruleanml import evaluation
from ceruleanml import preprocess
from fastai.data.block import DataBlock
from fastai.vision.data import ImageBlock, MaskBlock
from fastai.vision.augment import aug_transforms
from fastai.vision.learner import unet_learner
from fastai.data.transforms import IndexSplitter, Normalize
from fastai.metrics import DiceMulti
from ceruleanml.coco_load_fastai import record_collection_to_record_ids, get_image_path, record_to_mask
from torchvision.models import resnet18, resnet34, resnet50
from fastai.callback.fp16 import MixedPrecision
from fastai.callback.tensorboard import TensorBoardCallback
from datetime import datetime
from pathlib import Path
import os, random
from icevision.visualize import show_data
import torch

In [2]:
from fastai.callback.tracker import EarlyStoppingCallback, SaveModelCallback

### Parsing COCO Dataset with Icevision

In [3]:
class_map = {v: k for k, v in data.class_mapping_coco_inv.items()}
class_ints = list(range(1, len(list(class_map.keys())[:-1]) + 1))

In [4]:
data_path = "/root/"
mount_path = "../.."
train_set = "train-no-context-512"
tiled_images_folder_train = "tiled_images_no_context"
json_name_train = "instances_TiledCeruleanDatasetV2NoContextFiles.json"
val_set = "val-no-context-512"
tiled_images_folder_val= "tiled_images_no_context"
json_name_val = "instances_TiledCeruleanDatasetV2NoContextFiles.json"
coco_json_path_train = f"{mount_path}/partitions/{train_set}/{json_name_train}"
tiled_images_folder_train = f"{mount_path}/partitions/{train_set}/{tiled_images_folder_train}"
coco_json_path_val= f"{mount_path}/partitions/{val_set}/{json_name_val}"
tiled_images_folder_val = f"{mount_path}/partitions/{val_set}/{tiled_images_folder_val}"

class_map = {v: k for k, v in data.class_mapping_coco_inv.items()}
class_ints = list(range(1, len(list(class_map.keys())[:-1]) + 1))
negative_sample_count = 100
negative_sample_count_val = 50
mean = [60.73, 190.3, 4.3598]
std = [16.099, 17.846, 9.603]
area_thresh = 10
# f"{mount_path}/partitions/val/instances_tiled_cerulean_train_v2.json"

## looking at area distribution to find area threshold

In [5]:
df = preprocess.get_area_df(coco_json_path_train, tiled_images_folder_train)
df

  0%|          | 0/3863 [00:00<?, ?it/s]

,record_id,label,area
0,857,infra_slick,5918
1,4261,infra_slick,3366
2,4247,infra_slick,468
3,4248,infra_slick,326
4,7857,infra_slick,319
...,...,...,...
366,72482,ambiguous,145185
367,72483,ambiguous,2
368,72496,ambiguous,134272
369,72497,ambiguous,48306


In [5]:
record_collection_with_negative_small_filtered_train = preprocess.load_set_record_collection(
    coco_json_path_train, tiled_images_folder_train, area_thresh, negative_sample_count
)
record_ids_train = record_collection_to_record_ids(record_collection_with_negative_small_filtered_train)

  0%|          | 0/3863 [00:00<?, ?it/s]

100%|██████████| 3014/3014 [00:00<00:00, 116804.48it/s]


  0%|          | 0/100 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:122


  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
record_collection_with_negative_small_filtered_val = preprocess.load_set_record_collection(
    coco_json_path_val, tiled_images_folder_val, area_thresh, negative_sample_count_val
)
record_ids_val = record_collection_to_record_ids(record_collection_with_negative_small_filtered_val)

  0%|          | 0/352 [00:00<?, ?it/s]

100%|██████████| 266/266 [00:00<00:00, 102234.48it/s]


  0%|          | 0/50 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:122


  0%|          | 0/50 [00:00<?, ?it/s]

In [7]:
assert len(set(record_ids_train)) + len(set(record_ids_val)) == len(record_ids_train) + len(record_ids_val)

In [8]:
train_val_record_ids = record_ids_train + record_ids_val
combined_record_collection = record_collection_with_negative_small_filtered_train + record_collection_with_negative_small_filtered_val

In [9]:
def get_val_indices(combined_ids, val_ids):
    return list(range(len(combined_ids)))[-len(val_ids):]

In [11]:
#show_data.show_records(random.choices(combined_train_records, k=9), ncols=3)

### Constructing a FastAI DataBlock that uses parsed COCO Dataset from icevision parser and applies transformations

In [10]:
mean = [60.73,       190.3,      4.3598]
std = [16.099,      17.846,       9.603]

In [11]:
batch_transfms = [aug_transforms(),  Normalize.from_stats(mean,std)]

In [12]:
val_indices = get_val_indices(train_val_record_ids, record_ids_val)

In [13]:
def get_image_by_record_id(record_id):
    return get_image_path(combined_record_collection, record_id)

def get_mask_by_record_id(record_id):
    return record_to_mask(combined_record_collection, record_id)

#size = 64  # Progressive resizing could happen here
augs = aug_transforms(flip_vert=True, max_warp=0.1) #, size=size)
coco_seg_dblock = DataBlock(
    blocks=(ImageBlock, MaskBlock(codes=class_ints)),
    get_x=get_image_by_record_id,
    splitter=IndexSplitter(val_indices),
    get_y=get_mask_by_record_id,
    batch_tfms=[Normalize.from_stats(mean,std)],
    n_inp=1,
)

dls = coco_seg_dblock.dataloaders(source=train_val_record_ids, batch_size=5)

/root/miniconda3/envs/fastai2/lib/python3.9/site-packages/torch/_tensor.py:1051: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ret = func(*args, **kwargs)


### Fastai2 Trainer

In [14]:
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d_%b_%Y_%H_%M_%S")
experiment_dir =  Path(f'{mount_path}/experiments/cv2/'+timestampStr+'_fastai_unet/')
experiment_dir.mkdir(exist_ok=True)

In [15]:
arch = 18
archs = {18: resnet18, 34: resnet34, 50: resnet50}

In [29]:
cbs = [TensorBoardCallback(projector=False, trace_model=False), 
       SaveModelCallback(monitor="valid_loss", with_opt=True), 
       EarlyStoppingCallback(monitor='valid_loss', min_delta=0.01, patience=3)]
# learner.recorder.metric_names
learner = unet_learner(dls, archs[arch], metrics=[DiceMulti()], 
                       model_dir=experiment_dir, n_out = 7, 
                       cbs=cbs) # SaveModelCallback saves model when there is improvement
# lr = learner.lr_find()

In [33]:
# lr

SuggestedLRs(valley=0.0002754228771664202)

In [ ]:
learner.fine_tune(100, 2e-4, cbs=cbs)

epoch,train_loss,valid_loss,dice_multi,time
0,0.115410,0.093775,0.141748,05:22


Better model found at epoch 0 with valid_loss value: 0.09377501159906387.


epoch,train_loss,valid_loss,dice_multi,time


In [ ]:
validation = learner.validate()

In [ ]:
size=512
savename = f'test_1batch_{arch}_{size}_{round(validation[1],3)}.pt'

In [ ]:
from ceruleanml.inference import save_fastai_model_state_dict_and_tracing, load_tracing_model, test_tracing_model_one_batch, logits_to_classes

In [ ]:
state_dict_pth, tracing_model_gpu_pth, tracing_model_cpu_pth  = save_fastai_model_state_dict_and_tracing(learner, dls, savename, experiment_dir)

In [ ]:
import torch
experiment_dir = '/root/data/experiments/cv2/24_May_2022_01_49_56_fastai_unet/'
savename = "tracing_cpu_test_1batch_18_512_0.082.pt"
tracing_model = load_tracing_model(os.path.join(experiment_dir, savename))
out_batch_logits = test_tracing_model_one_batch(dls.to('cpu'), tracing_model)

In [ ]:
conf, classes = logits_to_classes(out_batch_logits)

In [ ]:
classes.shape

In [ ]:
conf.shape

In [ ]:
learn.show_results(max_n=4, figsize=(20,20), vmin=0, vmax=3)

Default path for tensorboard logs is `./runs/`

In [ ]:
f'{mount_path}/experiments/cv2/'

In [ ]:
!ls '/root/data/experiments/cv2/20_May_2022_19_29_39_fastai_unet'

In [ ]:
!ls './runs/'

Copy logs to appropriate exeriments folder in the mounted GCS volume.

In [ ]:
!cp -R './runs/' {modelpath}'/tensorboard/'

Now, run the following from anywhere with gcs authenticated:

In [ ]:
!tensorboard --logdir="./runs"


# Model Inference and Result Evaluation

In [ ]:
val_record_ids = record_collection_to_record_ids(valid_records)
pred_arrs = []
val_arrs = []
with learn.no_logging():
    for v, i in zip(range(len(val_record_ids[0:10])), val_record_ids[0:10]):
        v = valid_records[v].as_dict()
        v_masks = v['detection']['masks'][0].to_mask(v['common']['height'],v['common']['width']).data
        p = get_image_path(valid_records,i)
        arr = skio.imread(p)
        pred_arr = learn.predict(arr)
        pred_arr = pred_arr[0].cpu().detach().numpy()
        pred_arrs.append(pred_arr)
        val_arrs.append(v_masks)

In [ ]:
cm, f1 = eval.cm_f1(val_arrs, pred_arrs, 6, mount_path)

In [ ]:
learner = torch.load("/root/data/experiments/cv2/10_May_2022_18_02_59_fastai_unet/18_64_0.493.pkl")

In [ ]:
learner.predict??

In [ ]:
import skimage.io as skio
val_record_ids = record_collection_to_record_ids(valid_records)
pred_arrs = []
with learner.no_logging():
    for i in val_record_ids:
        p = get_image_path(valid_records,i)
        arr = skio.imread(p)
        pred_arr = learner.predict(arr)
        pred_arrs.append(pred_arr)

In [ ]:
# this results in vm dying, not just kernel crash
# coco_seg_dblock = DataBlock(
#     blocks=(ImageBlock, MaskBlock(codes=class_ints)),
#     get_x=get_image_by_record_id,
#     get_y=get_mask_by_record_id,
#     n_inp=1,
# )

# dls = coco_seg_dblock.dataloaders(source=record_ids, batch_size=5)


In [ ]:
result = learner.get_preds(dl=dls[0])

In [ ]:
learner.get_preds??

In [ ]:
len(pred_arrs)

In [ ]:
target_label,prediction_arr, activations = pred_arrs[0]

In [ ]:
skio.imshow(target_label.cpu().detach().numpy())

In [ ]:
skio.imshow(base_img.cpu().detach().numpy()[0])

In [ ]:
skio.imshow(base_img.cpu().detach().numpy()[1])

In [ ]:
skio.imshow(base_img.cpu().detach().numpy()[1])

In [ ]:
skio.imshow(base_img.cpu().detach().numpy())

In [ ]:
array([      60.73,       190.3,      4.3598]) # means
array([     16.099,      17.846,       9.603]) # stats